### 텍스트 분석 수행 프로세스
#### 20 뉴스그룹 데이터 세트를 이용, 텍스트 분류
- 특정 문서의 분류를 학습 데이터를 통해 학습해 모델을 생성한 뒤 이 학습 모델을 이용해 다른 문서의 분류를 예측
- 텍스트를 피처 벡터화로 변환, 희소 행렬로 만들고 로지스틱 회귀를 이용해 분류 수행
- Count 기반 과 TF-IDF 기반의 벡터화를 각각 적용, 성능 비교
- 피처 벡터화를 위한 파라미터와 GridSearchCV 기반의 하이퍼파라미터 튜닝을 일괄적으로 수행

In [1]:
# 텍스트 정규화
from sklearn.datasets import fetch_20newsgroups
news_data = fetch_20newsgroups(subset='all', random_state=156)
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [2]:
import pandas as pd
len(news_data.data)
pd.Series(news_data.target).value_counts()
# print(news_data.target_names)
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

In [3]:
# 순수한 텍스트만으로 구성된 기사 내용으로 어떤 뉴스그룹에 속하는지 분류
from sklearn.datasets import fetch_20newsgroups

# print(news_data.DESCR)
# The 20 newsgroups dataset comprises around 18000 newsgroups posts on
# 20 topics split in two subsets: one for training (or development)
# and the other one for testing (or for performance evaluation). The split
# between the train and test set is based upon a messages posted before
# and after a specific date.

# 텍스트 정규화
# 뉴스그룹 기사내용을 제외하고 다른 정보 제거
# 제목, 소속, 이메일 등 헤더와 푸터 정보들은 분류의 타겟 클래스 값과 유사할 수 있음
train_news = fetch_20newsgroups(subset='train', remove=('headers','footers','quotes'),
                  random_state=156)
X_train = train_news.data
y_train = train_news.target
test_news = fetch_20newsgroups(subset='test',remove=('header','footers','quotes'),
                              random_state=156)
X_test = test_news.data
y_test = test_news.target

print('학습데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data),
                                           len(test_news.data)))

학습데이터 크기 11314, 테스트 데이터 크기 7532


In [ ]:
import pandas as pd
import numpy as np
print(pd.Series(y_test).value_counts().sort_index())

In [21]:
# 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가
# Count Vectorization으로 feature extraction 변환 수행. 
from sklearn.feature_extraction.text import CountVectorizer
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train,y_train)
X_train_cnt_vect = cnt_vect.transform(X_train)
# 학습 데이터로 fit( )된 CountVectorizer를 반드시 이용하여 테스트 데이터 
# feature extraction 변환 수행(피처 개수가 동일해야 함)
X_test_cnt_vect = cnt_vect.transform(X_test)
print(X_train_cnt_vect.shape)
print(X_test_cnt_vect.shape)

(11314, 101631)
(7532, 101631)


In [22]:
# 로지스틱 회귀를 적용, 뉴스그룹에 대한 분류 예측

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train)
lr_pred = lr_clf.predict(X_test_cnt_vect)
print(accuracy_score(y_test, lr_pred))

0.6481678173127987


In [23]:
# TF-IDF가 단순 카운트 기반 보다 높은 예측 정확도를 제공
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
# TF-IDF Vectorization 적용하여 학습 데이터셋과 테스트 데이터 셋 변환.
tfidf_vect =TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)
# LogisticRegression을 이용하여 학습/예측/평가 수행.
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print(accuracy_score(y_test,pred))

0.710169941582581


In [24]:
# 텍스트 분석에서 ML 성능 향상 : 최적의 알고리즘 선택, 최상의 피처 전처리
# 텍스트 정규화, 피터 벡터화(TF-IDF 벡터화에 다양한 파라미터 적용)
# stop words 필터링을 추가하고 ngram을 기본(1,1)에서 (1,2)로 변경
# max_df : 너무 높은 빈도수를 가지는 단어 피처를 제외
# n_gram_range : 튜플 형태로 (범위 최솟값, 범위 최댓값)을 지정
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2),max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print(accuracy_score(y_test,pred))

0.7250398300584174


#### Q. Random Forest, SVM을 적용 뉴스그룹에 대한 분류 예측을 수행하세요.
svm_clf = svm.SVC(kernel='rbf')

In [10]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train_tfidf_vect,y_train)
rf_pred = rf_clf.predict(X_test_tfidf_vect)
print(accuracy_score(y_test,rf_pred))

0.6756505576208178


In [11]:
from sklearn import svm
svm_clf = svm.SVC(kernel='rbf',random_state=0)
svm_clf.fit(X_train_tfidf_vect,y_train)
svm_pred = svm_clf.predict(X_test_tfidf_vect)
print(accuracy_score(y_test, svm_pred))

0.692777482740308


In [19]:
print(news_data.target_names)
print(svm_pred[0:10])
print(y_test[0:10])
print(X_test[3])

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
[12 12 12  7  8  1 16  6  4  8]
[ 4 11  1  7  8  1 18  6  4  8]
From: jkjec@westminster.ac.uk (Shazad Barlas)
Subject: Re: MR2 - noisy engine.
Organization: University of Westminster
Lines: 10

Just a quick note on the nwe shape MR2s in the UK.... 

When they first came out here, there were 3 models. The base model had an 
auto box and engine from the CAMRY 2.0 !!! Well I recentyl found out that this 
model is no longer profitable for Toyota and have since scraped it. I've also
noticed that auto MR2s have depreciated a lot more than the next model up...


In [ ]:
# DT
from sklearn.tree import DecisionTreeClassifier
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train_tfidf_vect,y_train)
dt_pred = dt_clf(X_test_tfidf_vect)
print(accuracy_score(y_test,dt_pred))

In [ ]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train_tfidf_vect,y_train)
knn_pred = knn_clf.predict(X_test_tfidf_vect)
print(accuracy_score(y_test,knn_pred))

In [8]:
# 30분 이상 걸림
from sklearn.model_selection import GridSearchCV
# 최적 C 값 도출 튜닝 수행. CV는 3 Fold셋으로 설정.
params = {'C':[5,10]}
gcv_lr = GridSearchCV(lr_clf, param_grid=params, cv=3, \
                          scoring='accuracy',verbose = 1)
gcv_lr.fit(X_train_tfidf_vect, y_train)
print(gcv_lr.best_params_)
# 최적 C 값으로 학습된 grid_cv로 예측 수행하고 정확도 평가.
lr_pred = gcv_lr.predict(X_test_tfidf_vect)
print(accuracy_score(y_test, lr_pred))


Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 28.5min finished


{'C': 10}
0.7287573021773766


In [7]:
# Pipeline 사용 및 GridSearchCV와의 결합
from sklearn.pipeline import Pipeline
# TfidfVectorizer 객체를 tfidf_vect 객체명으로, LogisticRegression객체를 
# lr_clf 객체명으로 생성하는 Pipeline생성
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english',\
                                   ngram_range=(1,2),max_df=300)),\
    ('lr_clf', LogisticRegression(C=10))])
# 별도의 TfidfVectorizer객체의 fit_transform( )과 LogisticRegression의 fit(), 
# predict( )가 필요 없음. 
# pipeline의 fit( ) 과 predict( ) 만으로 한꺼번에 Feature Vectorization과 
# ML 학습/예측이 가능. 
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print(accuracy_score(y_test, pred))

0.7287573021773766


In [ ]:
# 30분 이상 소요
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression())
])
# 하이퍼파라미터명이 개게 변수명과 결합 : 피처 벡터화 객체 파라미터와 Estimator 객체의 하이퍼파라미터 구별하기 위함
params = {'tfidf_vect__ngram_range':[(1,1),(1,2),(1,3)],
         'tfidf_vect__max_df':[100,300,700],
         'lr_clf__C':[1,5,10]}

grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3, \
                           scoring = 'accuracy', verbose = 1)
grid_cv_pipe.fit(X_train,y_train)
print(grid_cv_pipe.best_params_, grid_cv_pripe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print(accuracy_score(y_test,pred))

#### 뉴스기사의 카테고리 판정 - 네이버 뉴스 2400개 수집

In [26]:
# tfidf.py : TF-IDF로 텍스트를 벡터로 변환하는 모듈
from konlpy.tag import Okt
import pickle
import numpy as np

# KoNLPy의 Okt객체 초기화 
okt = Okt()
# 전역 변수 
word_dic = {'_id': 0} # 단어 사전
dt_dic = {} # 문장 전체에서의 단어 출현 횟수
files = [] # 문서들을 저장할 리스트

def tokenize(text):
    '''KoNLPy로 형태소 분석하기''' 
    result = []
    word_s = okt.pos(text, norm=True, stem=True)
    for n, h in word_s:
        if not (h in ['Noun', 'Verb ', 'Adjective']): continue
        if h == 'Punctuation' and h2 == 'Number': continue
        result.append(n)
    return result

def words_to_ids(words, auto_add = True):
    ''' 단어를 ID로 변환하기 ''' 
    result = []
    for w in words:
        if w in word_dic:
            result.append(word_dic[w])
            continue
        elif auto_add:
            id = word_dic[w] = word_dic['_id']
            word_dic['_id'] += 1
            result.append(id)
    return result

def add_text(text):
    '''텍스트를 ID 리스트로 변환해서 추가하기''' 
    ids = words_to_ids(tokenize(text))
    files.append(ids)

def add_file(path):
    '''텍스트 파일을 학습 전용으로 추가하기''' 
    with open(path, "r", encoding="utf-8") as f:
        s = f.read()
        add_text(s)

def calc_files():
    '''추가한 파일 계산하기''' 
    global dt_dic
    result = []
    doc_count = len(files)
    dt_dic = {}
    # 단어 출현 횟수 세기
    for words in files:
        used_word = {}
        data = np.zeros(word_dic['_id'])
        for id in words:
            data[id] += 1
            used_word[id] = 1
        # 단어 t가 사용되고 있을 경우 dt_dic의 수를 1 더하기 
        for id in used_word:
            if not(id in dt_dic): dt_dic[id] = 0
            dt_dic[id] += 1
        # 정규화하기 
        data = data / len(words) 
        result.append(data)
    # TF-IDF 계산하기 
    for i, doc in enumerate(result):
        for id, v in enumerate(doc):
            idf = np.log(doc_count / dt_dic[id]) + 1
            doc[id] = min([doc[id] * idf, 1.0])
        result[i] = doc
    return result

def save_dic(fname):
    '''사전을 파일로 저장하기''' 
    pickle.dump(
        [word_dic, dt_dic, files],
        open(fname, "wb"))

def load_dic(fname):
    '''사전 파일 읽어 들이기''' 
    global word_dic, dt_dic, files
    n = pickle.load(open(fname, 'rb'))
    word_dic, dt_dic, files = n

def calc_text(text):
    ''' 문장을 벡터로 변환하기 ''' 
    data = np.zeros(word_dic['_id'])
    words = words_to_ids(tokenize(text), False)
    for w in words:
        data[w] += 1
    data = data / len(words)
    for id, v in enumerate(data):
        idf = np.log(len(files) / dt_dic[id]) + 1
        data[id] = min([data[id] * idf, 1.0])
    return data
# 모듈 테스트하기 
if __name__ == '__main__':
    add_text('비')
    add_text('오늘은 비가 내렸어요.') 
    add_text('오늘은 더웠지만 오후부터 비가 내렸다.') 
    add_text('비가 내리는 일요일이다.') 
    print(calc_files())
    print(word_dic)

[array([1., 0., 0., 0., 0.]), array([0.5       , 0.84657359, 0.        , 0.        , 0.        ]), array([0.25      , 0.4232868 , 0.59657359, 0.59657359, 0.        ]), array([0.5, 0. , 0. , 0. , 1. ])]
{'_id': 5, '비': 0, '오늘': 1, '덥다': 2, '오후': 3, '일요일': 4}


In [27]:
# 네이버 뉴스 정치, 경제, 생활, IT/과학 기사 TF-IDF 벡터 변환 후 genre.pickle로 저장
# 텍스트 분류 과정
# 1. 텍스트에서 불필요한 품사를 제거
# 2. 사전을 기반으로 단어를 숫자로 변환
# 3. 파일 내부의 단어 출현 비율을 계산
# 4. 데이터를 학습

# makedb_tfidf.py : 문장의 형태소를 벡터로 변환
import os, glob, pickle
import tfidf

# 변수 초기화
y = []
x = []

# 디렉터리 내부의 파일 목록 전체에 대해 처리하기 
def read_files(path, label):
    print("read_files=", path)
    files = glob.glob(path + "/*.txt")
    for f in files:
        if os.path.basename(f) == 'LICENSE.txt': continue
        tfidf.add_file(f)
        y.append(label)

# 기사를 넣은 디렉터리 읽어 들이기 
read_files('dataset/100', 0)
read_files('dataset/101', 1)
read_files('dataset/103', 2)
read_files('dataset/105', 3)


# TF-IDF 벡터로 변환하기 
x = tfidf.calc_files()

# 저장하기 
pickle.dump([y, x], open('dataset/genre.pickle', 'wb'))
tfidf.save_dic('dataset/genre-tdidf.dic')
print('ok')

read_files= dataset/100
read_files= dataset/101
read_files= dataset/103
read_files= dataset/105
ok


In [30]:
import pickle
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
import numpy as np

# TF-IDF 데이터베이스 읽어 들이기
data = pickle.load(open("dataset/genre.pickle", "rb"))
y = data[0] # 레이블
x = data[1] # TF-IDF

print(len(x))
print(len(y))
# 학습 전용과 테스트 전용으로 구분하기 
x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=0.2)

# 나이브 베이즈로 학습하기 
model = GaussianNB()
model.fit(x_train, y_train)

# 평가하고 결과 출력하기 
y_pred = model.predict(x_test)
acc = metrics.accuracy_score(y_test, y_pred)
rep = metrics.classification_report(y_test, y_pred)

print("정답률=", acc)
print(rep)


3197
3197
정답률= 0.7828125
              precision    recall  f1-score   support

           0       0.88      0.88      0.88       161
           1       0.84      0.62      0.71       177
           2       0.71      0.87      0.78       154
           3       0.72      0.79      0.75       148

    accuracy                           0.78       640
   macro avg       0.79      0.79      0.78       640
weighted avg       0.79      0.78      0.78       640



#### Q. genre.pickle 이름으로 데이터를 불러와서 LR, RF, DT, SVM, KNN 모델로 학습 및 펑가를 수행하고 정확도 및 Classification_report를 출력하세요.

In [ ]:
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
import numpy as np

# TF-IDF 데이터베이스 읽어 들이기
data = pickle.load(open("dataset/genre.pickle", "rb"))
y = data[0] # 레이블
x = data[1] # TF-IDF

# 학습 전용과 테스트 전용으로 구분하기 
x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=0.2)

model = LogisticRegression()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
acc = metrics.accuracy_score(y_test, y_pred)
rep = metrics.classification_report(y_test, y_pred)
print('정확도:', acc)
print(rep)